# Ce projet consiste a detecté les maladies des cultures


- on charge une image grâce à un bot sur Telegram
- Derrière le bot se trouve notre modèle
- Notre modèle doit analyser l'image
- Elle doit detecté si c'est une plante sinon une erreur sera envoyé
    - Sur l'interface de bot sur Telegram
    - Et de reprendre
- Si c'est une plante
    - si Il y a une maladie il envoie le nom de la maladie et les potentiels solutions
    - S'il n'y a pas une message encourangente sera envoyé par le bot
- Notre modèle sera hébergé sur Render

In [1]:


import tensorflow as tf
import json
from PIL import Image
import numpy as np
import tensorflow_hub as hub


In [3]:

# Chargement des donnees des maladies
with open('../json/disease.json', 'r', encoding='utf-8') as file:
    DISEASES = json.load(file)



In [5]:


# Download and load the latest version of the model

model = hub.load("https://kaggle.com/models/rishitdagli/plant-disease/frameworks/TensorFlow2/variations/plant-disease/versions/1")


if model:
    print("Modèle chargé avec succès")
    print(model)
else:
    print("Échec du chargement du modèle")


Modèle chargé avec succès
<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x000002B864408F50>


In [6]:


# Prédire la maladie de la plante
def predict_disease(image_path):
    try:
        # Charger et prétraiter l'image
        # Load the image and preprocess it
        img = Image.open('./'+image_path)
        print(img)
        img = img.resize((224, 224))
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
        img = tf.expand_dims(img, axis=0)

        # Make the prediction
        prediction = model(img)
        predicted_class = tf.argmax(prediction, axis=1)
        
        print(predicted_class)
        return int(predicted_class)

    except Exception as e:
        print(f"Erreur lors de la prédiction : {e}")
        return None



In [14]:
# Trouver la maladie
def find_disease(image_path):
    disease_info = {'name': "Maladie Inconnue", 'solution': "Aucune solution"}
    disease_id = predict_disease(image_path)

    if disease_id is not None:
        disease_info = DISEASES.get(str(disease_id), disease_info)

    return disease_info['name'], disease_info['solution']

In [15]:
find_disease('../image/temp_image.jpg')

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=256x256 at 0x2B84C80EE10>
tf.Tensor([36], shape=(1,), dtype=int64)


('Tomate___Virus de la mosaïque de la tomate',
 'Supprimer les plants malades et désinfecter les outils.')